# Shockwave Traffic Jam

## Brief
L'addestramento consiste nel miglioramento dei parametri del modello a seguito di un batch di simulazioni.

### Make the dataset
Ogni dato del dataset è ottenuto stabilendo casualmente:
1. Lunghezza della strada
2. Numero di veicoli
3. Parametri per **OVM**
4. Parametri per **FTL**
5. Bilanciamento tra **OVM** e **FTL**
Si crea così un dato senza addensamenti
Le velocità sono ottenute in accordo con il modello **OVM**.

### Data Augmentation
Si aumentano il numero di dati in modo casuale proponendo una perturbazione delle velocità e un tempo di simulazione iniziale prima di attivare l'apprendimento.

Ad ogni chiamata del training quindi ci sarà l'aggiunta di rumore.

### Training
Si addestra la rete con due sistemi di premi:
1. feedback ad ogni tempo $\tau$ che suggeriscono alla rete quanto il suo attuale andamento è gradevole (accelerazioni contenute):
   1. Si tratta di un fine-tuning degli ultimi layer (più istintivi).
   2. Il simulatore prenderà traccia dell'accelerazione più forte (in modulo).
2. premio finale al tempo $T$ che indica alla rete lo spazio percorso complessivo:
   1. Si aggiorna il ragionamento e la memoria della rete, si tratta di un tuning dei layer più superficiali.
   2. Il simulatore prenderà traccia dello spazio percorso totale dal veicolo autonomo.

## The model
Il modello non ha accesso completo a tutta la strada, tuttavia può ricordare quello che vede e quello che ha fatto.

In questo modo in una strada lineare semplicemente il modello assocerà ciò che vede ad un'esperienza antecedente e anche se non è vero che quanto farà lo aiuterà, agirà comunque come se fosse per il bene proprio.

Per questa ragione usiamo delle RNN per l'addestramento di alto livello, semplicemente la rete memorizzerà la qualità della strategia nelle RNN

L'interpretazione delle strategie sarà fatta a livelli più profondi che ne stimano l'applicazione e che saranno addestrati con fine-tuning

# Make the dataset

Il dataset generato è solo una radice di costruzione del vero dataset usato per l'addestramento.

Generiamo quindi questa radice che prende in input:
- range di valori per la densità
- range di valori per $d_0$ richiesto da OVM
- range di valori per $\Delta$

La radice propone una distribuzione uniforme dei veicoli in uno stato stazionario.

In [1]:
# MAKE DATASET ROOT
from source import ShockwaveTrafficJam
import torch
import os

dataset_path = 'data/db/TRAFFIC'
os.makedirs(dataset_path, exist_ok=True)

ShockwaveTrafficJam.dataset.make_traffic(1_000_000, torch.device('cuda'), 'data/db/TRAFFIC')

Il dataset è quindi un tensore avente 3 colonne indicanti:
- densità
- d0_OVM
- Delta

In [2]:
# LOAD DATASET ROOT
from source import ShockwaveTrafficJam

dataset = ShockwaveTrafficJam.dataset.Dataset(dataset_path, root=True).to(torch.device('cuda'))

Il dataloader carica la radice e viene usato per produrre facilmente i batch.

Ogni batch presenta un'informazione completa e parallelizzata:
- posizioni dei veicoli : tensore di shape (batch_size, n_vehicles)
- velocità dei veicoli : tensore di shape (batch_size, n_vehicles)
- len_road, d0_OVM, Delta, Vmax, tau, d0_FTL, gamma, beta : tensore di shape (batch_size, 8)

In particolare sarà necessario passare al dataloader:
- range per il numero di veicoli : usato per generare il numero di veicoli del batch
- range per Vmax, tau, d0_FTL, gamma, beta
  
Sarà inoltre aggiunto un trasformatore dei dati che fungerà da data augmentation.

In [3]:
# CREATE DATALOADER ROOT
from source import ShockwaveTrafficJam

transformer = ShockwaveTrafficJam.transforms.Compose([
    ShockwaveTrafficJam.transforms.RandomDropout(0.1),                # 10% of the data will be dropped
    ShockwaveTrafficJam.transforms.RandomNoise(0.1, 1.2),             # perturbation of position and speed
    ShockwaveTrafficJam.transforms.Simulation(0.0005, 60_000),        # simulate the traffic jam (30 sec) with dt=0.0005
])

dataloader = ShockwaveTrafficJam.dataloader.DataLoader(dataset, batch_size=16_384, shuffle=True, transform=transformer, root=True).to(torch.device('cuda'))

Uso il dataloader per creare il dataset completo che sarà poi usato per l'addestramento

In [4]:
# make the dataset
from tqdm.notebook import tqdm

for i, sim in tqdm(enumerate(dataloader), total=len(dataloader)):
    sim = sim.to(torch.device('cpu'))
    sim.save(i, 'data/db/TRAFFIC')

  0%|          | 0/62 [00:00<?, ?it/s]

# Open dataset

E' possibile aprire direttamente il nuovo dataset con tutti gli scenari costruiti.

Ogni file rappresenta una parte del dataset, il dataloader carica i file in RAM e poi li smaltisce per l'addestramento.



In [ ]:
# LOAD the dataset
from source import ShockwaveTrafficJam

# libero la gpu da ogni variabile di torch
torch.cuda.empty_cache()

dataset = ShockwaveTrafficJam.dataset.Dataset('data/db/TRAFFIC', root=False).to(torch.device('cuda'))
dataloader = ShockwaveTrafficJam.dataloader.DataLoader(dataset, batch_size=256, shuffle=True).to(torch.device('cuda'))

La rete è definita strutturalmente come segue:
1.  una CNN 1D a 3 canali che prende in input le differenze di velocità, la velocità e la distanza
2.  un trasformatore che recupera la serie temporale e la elabora (self attention o con RNN)???
3.  un